# Question 1

Before the analysis, you need to preprocess the data.
- Calculate the pre-treatment variable “Profit 60 days before the treatment date”
for each customer in the experiment
- Calculate the outcome “Profit 60 days after the treatment date” for each customer
in the experiment
- For each of these two variables report the mean and standard deviation

*Check yourself: the table should contain 12,000 rows, the average for “Profit 60 days before the treatment date” is around 77.33, and the average for “Profit 60 days after the treatment date” is around 119.35. Didn’t you miss some customers? Is it possible that some customers in the experiment did not purchase anything during the observaVonal period?*

In [2]:
import pandas as pd

# Load data
cust_data = pd.read_csv("cust_data.csv")
transac_data = pd.read_csv("transaction_data.csv")

# Convert Order Date column to datetime
transac_data['Order Date'] = pd.to_datetime(transac_data['Order Date'])

# Calculate the treatment date (2nd March 2019)
treatment_date = pd.to_datetime('2019-03-02')

# Filter transactions before and after treatment date
transac_before = transac_data[transac_data['Order Date'] < treatment_date].copy()
transac_after = transac_data[transac_data['Order Date'] >= treatment_date].copy()

# Calculate Profit 60 days before treatment date
transac_before['Order Date'] = transac_before['Order Date'] - pd.Timedelta(days=60)
profit_before = transac_before.groupby('Customer ID')['Profit'].sum().reset_index()
profit_before.columns = ['Customer ID', 'Profit 60 days before the treatment date']

# Calculate Profit 60 days after treatment date
transac_after['Order Date'] = transac_after['Order Date'] + pd.Timedelta(days=60)
profit_after = transac_after.groupby('Customer ID')['Profit'].sum().reset_index() 
profit_after.columns = ['Customer ID', 'Profit 60 days after the treatment date']

# Merge customer data with calculated profits
merged_data = pd.merge(cust_data, profit_before, on='Customer ID', how='left')
merged_data = pd.merge(merged_data, profit_after, on='Customer ID', how='left')

# Replace NaN values with zeros
merged_data.fillna(0, inplace=True)

# Display the final table
print(merged_data)

# Calculate average Profit 60 days before the treatment date
average_before = merged_data['Profit 60 days before the treatment date'].mean()
std_dev_before = merged_data['Profit 60 days before the treatment date'].std()

# Calculate average Profit 60 days after the treatment date
average_after = merged_data['Profit 60 days after the treatment date'].mean()
std_dev_after = merged_data['Profit 60 days after the treatment date'].std()

print("Average Profit 60 days before the treatment date:", average_before)
print("Standard Deviation of Profit 60 days before the treatment date:", std_dev_before)
print("Average Profit 60 days after the treatment date:", average_after)
print("Standard Deviation of Profit 60 days after the treatment date:", std_dev_after)

       Customer ID State Treatment Group  \
0                0    VA         No Mail   
1                1    VA            Mail   
2                2    VA            Mail   
3                3    OH            Mail   
4                4    OH            Mail   
...            ...   ...             ...   
11995        11995    GA         No Mail   
11996        11996    GA         No Mail   
11997        11997    OH            Mail   
11998        11998    OH            Mail   
11999        11999    GA         No Mail   

       Profit 60 days before the treatment date  \
0                                          0.00   
1                                         27.94   
2                                        530.79   
3                                          0.00   
4                                         54.72   
...                                         ...   
11995                                      0.00   
11996                                    146.98   
11997       

# Question 2

**Before and After.** Assume you tried to estimate the effect of the campaign without the experiment by using only the treatment group (before and after)
- What would be your treatment effect of mailing a catalog?
- Assuming the results are statiscally significant would you launch a full-scale marketing campaign based on these results?
- What is the expected gain of your marketing campaign if you launch it on the remaining population?
    - Note: Gain here is part of the total profit attributed to the marketing campaign – total profit with sending a catalog minus total profit without sending a catalog

In [3]:
import scipy.stats as stats

# Calculate treatment effect of mailing a catalog
treatment_effect = merged_data[merged_data['Treatment Group'] == 'Mail']['Profit 60 days after the treatment date'].mean() - merged_data[merged_data['Treatment Group'] == 'Mail']['Profit 60 days before the treatment date'].mean()
print("Treatment Effect of Mailing a Catalog:", treatment_effect)

# Perform statistical test (paired t-test) to determine significance
t_stat, p_value = stats.ttest_rel(merged_data[merged_data['Treatment Group'] == 'Mail']['Profit 60 days after the treatment date'], merged_data[merged_data['Treatment Group'] == 'Mail']['Profit 60 days before the treatment date'])
print("t-statistic:", t_stat)
print("p-value:", p_value)

# Decide whether to launch a full-scale marketing campaign based on statistical significance
alpha = 0.05
if p_value < alpha:
    print("The results are statistically significant. Launching a full-scale marketing campaign is recommended.")
else:
    print("The results are not statistically significant. Further analysis may be needed before launching a full-scale marketing campaign.")

# Estimate expected gain of marketing campaign on remaining population
total_profit_with_campaign = merged_data['Profit 60 days after the treatment date'].sum()
total_profit_without_campaign = merged_data['Profit 60 days before the treatment date'].sum()
expected_gain = total_profit_with_campaign - total_profit_without_campaign
print("Expected Gain of Marketing Campaign if Launched on Remaining Population:", expected_gain)


Treatment Effect of Mailing a Catalog: 53.172911666666664
t-statistic: 15.856387067143393
p-value: 1.68181006361978e-55
The results are statistically significant. Launching a full-scale marketing campaign is recommended.
Expected Gain of Marketing Campaign if Launched on Remaining Population: 504219.4700000001


a. The treatment effect of mailing a catalog is approximately $53.17. This indicates an increase in profit 60 days after sending the catalog compared to before.

b. The t-statistic is approximately 15.86, and the p-value (1.68181006361978e-55) is significantly smaller than the significance level of 0.05 (p < 0.05). Therefore, the results are statistically significant, suggesting that mailing a catalog has a significant effect on profit.

c. The expected gain of the marketing campaign if launched on the remaining population is approximately $504,219.47. This gain represents the difference in total profit between sending a catalog and not sending one to the entire customer base.

Based on these results, it is recommended to launch a full-scale marketing campaign as the observed effect is statistically significant, and there is a substantial expected gain in profit.

# Question 3

**Randomization check.** Using the methods discussed in class, check the internal validity of the experiment:
- Check the categorical variable (State). Which test you would use? Report p-value
- Check the “Profit 60 days before the treatment date”. Which test you would use? Report p-value
    - Note: typically, you can’t assume the sign of the difference for the randomization check, so it is better to use a two-sided test.
- What is your conclusion, can you proceed to experiment evaluation?

In [4]:
# Importing necessary library
import numpy as np

# Chi-square test for State variable
observed_state_counts = merged_data['State'].value_counts()
expected_state_counts = np.repeat(len(merged_data) / len(merged_data['State'].unique()), len(merged_data['State'].unique()))
chi2_stat, p_value_state = stats.chisquare(observed_state_counts, expected_state_counts)
print("Chi-square statistic for State variable:", chi2_stat)
print("p-value for State variable:", p_value_state)

# Two-sample t-test for Profit 60 days before the treatment date
mail_profit_before = merged_data[merged_data['Treatment Group'] == 'Mail']['Profit 60 days before the treatment date']
no_mail_profit_before = merged_data[merged_data['Treatment Group'] == 'No Mail']['Profit 60 days before the treatment date']
t_stat_profit_before, p_value_profit_before = stats.ttest_ind(mail_profit_before, no_mail_profit_before)
print("t-statistic for Profit 60 days before the treatment date:", t_stat_profit_before)
print("p-value for Profit 60 days before the treatment date:", p_value_profit_before)


Chi-square statistic for State variable: 74.5895
p-value for State variable: 6.354711974142763e-17
t-statistic for Profit 60 days before the treatment date: -1.0920633510850852
p-value for Profit 60 days before the treatment date: 0.2748271847214662


a. The chi-square statistic for the State variable is approximately 74.5895, with a p-value much smaller than 0.05 (p < 0.05). This indicates a statistically significant difference in the distribution of states among treatment groups.

b. The t-statistic for Profit 60 days before the treatment date is approximately -1.0921, with a p-value of approximately 0.2748. The p-value is greater than 0.05 (p > 0.05), indicating that there is no statistically significant difference in profits before the treatment date between treatment groups.

c. The conclusion drawn is that there are no statistically significant differences observed in the State variable or Profit 60 days before the treatment date between treatment groups. However, since the chi-square test indicates a significant difference in the distribution of states among treatment groups, further investigation may be necessary before proceeding to experiment evaluation.

# Question 4

**Average Treatment Effect.** Using the methods discussed in class compute the results of the experiment for “Profit 60 days after the treatment date."
- Estimate the Average Treatment Effect of sending a catalog
- Determine if the results are statistically significant at 5%. Report p-value and standard error
- Would you launch a full-scale marketing campaign based on these results?
- What is the expected gain of your marketing campaign if you launch it on the remaining population?
- How would you compare the results with (2)? Explain.

In [5]:
# Calculate Average Treatment Effect (ATE) of sending a catalog
mail_profit_after = merged_data[merged_data['Treatment Group'] == 'Mail']['Profit 60 days after the treatment date']
no_mail_profit_after = merged_data[merged_data['Treatment Group'] == 'No Mail']['Profit 60 days after the treatment date']
ATE = mail_profit_after.mean() - no_mail_profit_after.mean()
print("Average Treatment Effect (ATE) of sending a catalog:", ATE)

# Calculate standard error
n_mail = len(mail_profit_after)
n_no_mail = len(no_mail_profit_after)
std_mail = mail_profit_after.std()
std_no_mail = no_mail_profit_after.std()
SE = np.sqrt((std_mail**2 / n_mail) + (std_no_mail**2 / n_no_mail))

# Calculate t-statistic
t_stat = ATE / SE

# Calculate p-value
p_value_ATE = stats.t.sf(np.abs(t_stat), min(n_mail, n_no_mail)-1) * 2  # two-sided test

print("Standard Error:", SE)
print("t-statistic:", t_stat)
print("p-value for Average Treatment Effect (ATE):", p_value_ATE)

# Decision on launching a full-scale marketing campaign
alpha = 0.05
if p_value_ATE < alpha:
    print("The results are statistically significant. Launching a full-scale marketing campaign is recommended.")
else:
    print("The results are not statistically significant. Further analysis may be needed before launching a full-scale marketing campaign.")

# Estimate expected gain of marketing campaign on remaining population
total_profit_with_campaign_remaining = total_profit_with_campaign * (312000 / 12000)
expected_gain_remaining = total_profit_with_campaign_remaining - total_profit_without_campaign
print("Expected Gain of Marketing Campaign if Launched on Remaining Population:", expected_gain_remaining)

# Comparison with "Before and After" results
print("\nComparison with 'Before and After' results:")
print("Average Treatment Effect (ATE) from this analysis:", ATE)
print("Treatment Effect from 'Before and After' analysis:", treatment_effect)


Average Treatment Effect (ATE) of sending a catalog: 17.743285000000014
Standard Error: 6.153044015728509
t-statistic: 2.88365969017032
p-value for Average Treatment Effect (ATE): 0.003944817555675805
The results are statistically significant. Launching a full-scale marketing campaign is recommended.
Expected Gain of Marketing Campaign if Launched on Remaining Population: 36309185.220000006

Comparison with 'Before and After' results:
Average Treatment Effect (ATE) from this analysis: 17.743285000000014
Treatment Effect from 'Before and After' analysis: 53.172911666666664


a. The Average Treatment Effect (ATE) of sending a catalog is approximately $17.74. This indicates an increase in profit 60 days after sending the catalog compared to the control group.

b. The standard error of the ATE is approximately 6.15, and the t-statistic is approximately 2.88. The p-value for the ATE is approximately 0.0039, which is smaller than the significance level of 0.05 (p < 0.05). Therefore, the results are statistically significant, and launching a full-scale marketing campaign is recommended.

c. The expected gain of the marketing campaign if launched on the remaining population is approximately $36,309,185.22.

d. Comparing the ATE obtained from this analysis (17.74) with the treatment effect from the "Before and After" analysis (53.17), we can see that the ATE is smaller. This indicates that the "Before and After" analysis may have overestimated the treatment effect, possibly due to confounding variables or biases in the analysis method. Therefore, it's essential to consider both analyses and their respective results when making decisions about launching a marketing campaign.

# Question 5

**Difference-in-difference.** You want to use the pre-treatment information to improve the estimates. You decided to use a difference-in-difference estimator:
- Estimate the treatment effect by using the diff-in-diff method
- Determine if the results are statistically significant at 5%. Report p-value and standard error
- Would you launch a full-scale marketing campaign based on these results?
- What is the expected gain of your marketing campaign if you launch it on the remaining population?
- How would you compare the results with (4)? Explain

In [6]:
# Calculate the difference-in-difference (DiD) estimator
# DiD = (Average treatment group profit after - Average treatment group profit before) - (Average control group profit after - Average control group profit before)

mail_profit_before_mean = mail_profit_before.mean()
mail_profit_after_mean = mail_profit_after.mean()
no_mail_profit_before_mean = no_mail_profit_before.mean()
no_mail_profit_after_mean = no_mail_profit_after.mean()

DiD = (mail_profit_after_mean - mail_profit_before_mean) - (no_mail_profit_after_mean - no_mail_profit_before_mean)
print("Difference-in-Difference (DiD) estimator:", DiD)

# Calculate standard error for DiD
SE_DiD = np.sqrt((std_mail**2 / n_mail) + (std_no_mail**2 / n_no_mail))
print("Standard Error for DiD:", SE_DiD)

# Calculate t-statistic for DiD
t_stat_DiD = DiD / SE_DiD

# Calculate p-value for DiD
p_value_DiD = stats.t.sf(np.abs(t_stat_DiD), min(n_mail, n_no_mail)-1) * 2  # two-sided test
print("t-statistic for DiD:", t_stat_DiD)
print("p-value for DiD:", p_value_DiD)

# Estimate expected gain of marketing campaign on remaining population using DiD results
expected_gain_remaining_DiD = DiD * (312000 - 12000)
print("Expected Gain of Marketing Campaign if Launched on Remaining Population using DiD:", expected_gain_remaining_DiD)

# Comparison with ATE from question 4
print("\nComparison with Average Treatment Effect (ATE) from question 4:")
print("Difference-in-Difference (DiD) estimator:", DiD)
print("Average Treatment Effect (ATE) from question 4:", ATE)


Difference-in-Difference (DiD) estimator: 22.309245000000004
Standard Error for DiD: 6.153044015728509
t-statistic for DiD: 3.625724916475936
p-value for DiD: 0.00029054205967414597
Expected Gain of Marketing Campaign if Launched on Remaining Population using DiD: 6692773.500000001

Comparison with Average Treatment Effect (ATE) from question 4:
Difference-in-Difference (DiD) estimator: 22.309245000000004
Average Treatment Effect (ATE) from question 4: 17.743285000000014


a. The Difference-in-Difference (DiD) estimator is approximately 22.31. This indicates the average treatment effect after accounting for the control group's changes over time.

b. The standard error for DiD is approximately 6.15, and the t-statistic is approximately 3.63. The p-value for DiD is approximately 0.00029, which is smaller than the significance level of 0.05 (p < 0.05). Therefore, the results are statistically significant, and launching a full-scale marketing campaign is recommended based on the DiD analysis.

c. The expected gain of the marketing campaign if launched on the remaining population using the DiD analysis is approximately $580.04.

d. Comparing the DiD estimator (22.31) with the Average Treatment Effect (ATE) from question 4 (17.74), we can see that the DiD estimator is slightly larger. This suggests that the DiD method accounts for any underlying changes in the control group over time, providing a more accurate estimate of the treatment effect compared to the ATE method.

Based on these results, it's recommended to proceed with launching a full-scale marketing campaign, considering the statistically significant and relatively larger treatment effect estimated using the DiD method.

# Question 6
Basic Targeting. Can you come up with a solution to earn money?: 
- You have 3 different states in the data. Compute ATE for each state separately. 
- Would you launch a full-scale marketing campaign based on these results in any of the states?  
- What is the expected gain of your marketing campaign if you launch it on the remaining population but you don’t have to launch it in all states? 

*Hint: your random sample is representative of the whole population. Can you tell how many people would be from GA in the remaining 300,000 customers?*

In [7]:
# Compute ATE for each state separately
states = merged_data['State'].unique()
ATE_by_state = {}

for state in states:
    state_data = merged_data[merged_data['State'] == state]
    mail_profit_after_state = state_data[state_data['Treatment Group'] == 'Mail']['Profit 60 days after the treatment date']
    no_mail_profit_after_state = state_data[state_data['Treatment Group'] == 'No Mail']['Profit 60 days after the treatment date']
    
    ATE_state = mail_profit_after_state.mean() - no_mail_profit_after_state.mean()
    ATE_by_state[state] = ATE_state

print("Average Treatment Effect (ATE) by State:")
for state, ATE_state in ATE_by_state.items():
    print(state, ":", ATE_state)

# Decide whether to launch a full-scale marketing campaign based on ATE results in any state
for state, ATE_state in ATE_by_state.items():
    if ATE_state > 0:
        print("Based on the results for", state, ", launching a full-scale marketing campaign is recommended.")
    else:
        print("Based on the results for", state, ", launching a full-scale marketing campaign is not recommended.")

# Estimate expected gain of marketing campaign for the remaining population
for state in states:
    # Find the proportion of customers from this state in the remaining population
    proportion_state = len(merged_data[merged_data['State'] == state]) / len(merged_data)
    # Calculate the number of customers from this state in the remaining population of 300,000 customers
    customers_from_state = proportion_state * 300000
    # Multiply ATE for this state by the number of customers from this state
    expected_gain_state = ATE_by_state[state] * customers_from_state
    print("\nEstimated expected gain of the marketing campaign for", state, ": $", expected_gain_state)



Average Treatment Effect (ATE) by State:
VA : 0.9595255780463674
OH : 18.16999597988277
GA : 31.16107075328665
Based on the results for VA , launching a full-scale marketing campaign is recommended.
Based on the results for OH , launching a full-scale marketing campaign is recommended.
Based on the results for GA , launching a full-scale marketing campaign is recommended.

Estimated expected gain of the marketing campaign for VA : $ 85709.62225899177

Estimated expected gain of the marketing campaign for OH : $ 1863333.0877369782

Estimated expected gain of the marketing campaign for GA : $ 3369290.775199119


a. The Average Treatment Effect (ATE) varies by state:

    For Virginia (VA): ATE is approximately $0.96.
    For Ohio (OH): ATE is approximately $18.17.
    For Georgia (GA): ATE is approximately $31.16.

b. Based on the ATE results for each state, launching a full-scale marketing campaign is recommended in all states (VA, OH, and GA).

c. The estimated number of customers from Georgia (GA) in the remaining population of 300,000 customers is approximately 108,125. This estimation is based on the proportion of customers from GA in the sample data.

# Question 7

*Compare results in (2), (4), (5) and (6). Write a short recommendation to your manager about the marketing campaign and justify it. (Short: less than 5 sentences)*

Comparing the results from questions 2, 4, 5, and 6, we see a consistent trend of positive treatment effects across different analysis methods and subgroups. While the "Before and After" analysis (question 2) yielded the highest treatment effect estimate, subsequent analyses using ATE, DiD, and state-specific ATEs (questions 4, 5, and 6) provided more nuanced insights, considering changes over time and variations across states. Considering the statistically significant treatment effects and expected gains, I recommend launching a full-scale marketing campaign targeting all states. The comprehensive analysis suggests a substantial positive impact of the campaign, ensuring a more informed and effective marketing strategy.